In [1]:
from soynlp.word import WordExtractor
from soynlp import DoublespaceLineCorpus
import pandas as pd



In [2]:
# 데이터 불러오기
data = pd.read_csv('./youtube_channel_comments_cleaneddata.csv')



In [3]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56405 entries, 0 to 56404
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   channelId            56405 non-null  object
 1   channelTitle         56405 non-null  object
 2   channelPublishedAt   56405 non-null  object
 3   subscriberCount      56405 non-null  int64 
 4   videoId              56405 non-null  object
 5   videoTitle           56405 non-null  object
 6   videoAuthorId        56405 non-null  object
 7   videoPublishedAt     56405 non-null  object
 8   duration             56405 non-null  int64 
 9   viewCount            56405 non-null  int64 
 10  likeCount            56405 non-null  int64 
 11  dislikeCount         56405 non-null  int64 
 12  commentId            56405 non-null  object
 13  commentAuthor        56277 non-null  object
 14  authorId             56405 non-null  object
 15  commentText          56405 non-null  object
 16  comm

In [4]:
# cleaned_commentText의 데이터 보기
data['cleaned_commentText']


0                야채 많이 드시고 가끔 요구르트 챙겨드시면 어느 정도 예방됨
1                  제일 확실한 건 변비 설사 반복 됩니다 꼭 검사해 보셔요
2                                      좋은 정보 감사합니다
3                                걸리면 하느님 만나로 올라갈게요
4                         이 중에 하나도 해당 안 됐는데 대장암 4기
                           ...                    
56400     이지약사님 정말 오랜만에 뵙네요 그동안 잘 지내셨나요 너무 보고 싶었어요
56401                                 감사합니다 잘 지내셨죠
56402    이지 약사님 정말 오랜만에 뵙네요 그동안 잘 지내셨나요 너무 보고 싶었어요
56403        안녕하세요 감사합니다ㅎㅎ 부활해 보았습니 당ㅎㅎ 무더위 잘 보내셨죠
56404            ezyaksa 이지 약사님을 이렇게 뵈니 너무 반갑네요 ㅎㅎ
Name: cleaned_commentText, Length: 56405, dtype: object

In [6]:
# NaN 값 제거
data = data.dropna(subset=['cleaned_commentText'])

# 또는

# NaN 값 공백 문자열로 대체
data['cleaned_commentText'] = data['cleaned_commentText'].fillna('')

# soynlp 훈련
word_extractor = WordExtractor()
word_extractor.train(data['cleaned_commentText'])
word_score_table = word_extractor.extract()

training was done. used memory 0.416 Gbory 0.235 Gb
all cohesion probabilities was computed. # words = 40135
all branching entropies was computed # words = 56458
all accessor variety was computed # words = 56458


### 1. soynlp 토크나이저 구축
soynlp의 WordExtractor로부터 얻은 word_score_table을 이용하여 LTokenizer 또는 MaxScoreTokenizer를 구성할 수 있습니다.

In [7]:
from soynlp.tokenizer import LTokenizer

scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
tokenizer = LTokenizer(scores=scores)

### 2. 텍스트 토크나이징
토크나이저를 사용하여 텍스트 데이터를 토큰화합니다.



In [8]:
data['tokenized'] = data['cleaned_commentText'].apply(tokenizer.tokenize)

In [16]:
data['tokenized'] 

0        [야채, 많이, 드시, 고, 가끔, 요구르트, 챙겨, 드시면, 어느, 정도, 예방, 됨]
1         [제일, 확실, 한, 건, 변비, 설사, 반복, 됩니, 다, 꼭, 검사, 해, 보셔요]
2                                        [좋은, 정보, 감사, 합니다]
3                               [걸리, 면, 하느님, 만나로, 올라, 갈게요]
4                      [이, 중에, 하나, 도, 해당, 안, 됐는데, 대장암, 4기]
                               ...                        
56400    [이지약사님, 정말, 오랜만에, 뵙네요, 그동안, 잘, 지내셨나요, 너무, 보고, ...
56401                                   [감사, 합니다, 잘, 지내셨죠]
56402    [이지, 약사님, 정말, 오랜만에, 뵙네요, 그동안, 잘, 지내셨나요, 너무, 보고...
56403     [안녕하세요, 감사, 합니다ㅎㅎ, 부활해, 보았습니, 당ㅎㅎ, 무더위, 잘, 보내셨죠]
56404         [ezyaksa, 이지, 약사님, 을, 이렇게, 뵈니, 너무, 반갑네요, ㅎㅎ]
Name: tokenized, Length: 56146, dtype: object

In [ ]:
# 한글자, 지시대명사 등 제거
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if len(item) > 1])

# 한글자 이외의 형용사, 부사, 동사 불용어 stopword 목록 작성하여 반영
stopword = ['ㅎㅎ', 'ㅋㅋ', 'ㅠㅠ', 'ㅜㅜ', 'ㅇㅇ', 'ㅈㅈ', 'ㅊㅊ', 'ㅋㅋ', 'ㅎㅎ', 'ㅇㅇ', 'ㅈㅈ', 'ㅊㅊ']
data['tokenized'] = data['tokenized'].apply(lambda x: [item for item in x if item not in stopword])


### 3. 문서-단어 행렬 생성
scikit-learn의 CountVectorizer를 사용하여 문서-단어 행렬을 생성합니다. 이때, 토크나이저로 soynlp의 결과를 사용합니다.



In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# CountVectorizer 초기화
vectorizer = CountVectorizer(tokenizer=lambda x: x, lowercase=False)

# 문서-단어 행렬 생성
document_term_matrix = vectorizer.fit_transform(data['tokenized'])

/home/arkwith/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


### 4. LDA 모델 학습
scikit-learn의 LatentDirichletAllocation을 사용하여 LDA 모델을 학습합니다.


In [14]:
from sklearn.decomposition import LatentDirichletAllocation

# LDA 모델 초기화 및 학습
lda_model = LatentDirichletAllocation(n_components=10, random_state=0)
lda_model.fit(document_term_matrix)

LatentDirichletAllocation(random_state=0)

### 5. 결과 확인
학습된 LDA 모델로부터 각 토픽의 주요 단어들을 확인합니다.

In [15]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)

n_top_words = 10
tf_feature_names = vectorizer.get_feature_names_out()
print_top_words(lda_model, tf_feature_names, n_top_words)

Topic #0: 이 요 119 yakstory 는 게 을 가 다 은
Topic #1: 비타민 복용 이 먹고 을 같이 먹어도 마그네슘 섭취 영양제
Topic #2: 커피 가 도 요 믹스 먹고 ㅋㅋ 이 한 서
Topic #3: 섭취 괜찮 네 습니다 하셔도 다 하시는 게 에 제품
Topic #4: 이 안 영양제 가 도움 요 도 추천 에 은
Topic #5: 감사 합니다 https 시청 해주셔서 be youtu 1등은 정보 좋은
Topic #6: 수 이 말씀 을 도움 가 는 이건 에 의
Topic #7: 영상 잘 약사님 보고 이 너무 좋은 도 감사 에
Topic #8: 제품 가 거 이 문제 는 구매 요 은 같습니다
Topic #9: 도 안 가 이 거 다 한 사람 는 게


### 3. 채널별 데이터 준비
채널별로 데이터를 그룹화합니다. 예를 들어, 각 채널의 댓글을 하나의 문서로 간주할 수 있습니다.


In [9]:
grouped_data = data.groupby('channelId').agg({'tokenized': 'sum'})

4. 토픽 모델링
토픽 모델링을 위해 LDA(Latent Dirichlet Allocation)와 같은 알고리즘을 사용할 수 있습니다. gensim 라이브러리를 사용하여 LDA 모델을 구축할 수 있습니다.

In [12]:
from gensim import corpora, models

# # 사전 생성
# dictionary = corpora.Dictionary(grouped_data['tokenized'])

# # 코퍼스 생성
# corpus = [dictionary.doc2bow(text) for text in grouped_data['tokenized']]

# # LDA 모델 학습
# lda_model = models.LdaModel(corpus, num_topics=10, id2word=dictionary, passes=15)

ImportError: cannot import name 'triu' from 'scipy.linalg' (/home/arkwith/.local/lib/python3.10/site-packages/scipy/linalg/__init__.py)